# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa394b1ca60>
├── 'a' --> tensor([[-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116]])
└── 'x' --> <FastTreeValue 0x7fa394b1cb20>
    └── 'c' --> tensor([[-2.3509,  0.3106, -0.5697, -0.8740],
                        [ 0.4058, -0.3023, -0.5851, -0.6074],
                        [-0.6653, -0.5996, -1.4165,  1.1686]])

In [4]:
t.a

tensor([[-0.2846,  0.5964, -1.3886],
        [-1.2387, -0.3852,  0.3116]])

In [5]:
%timeit t.a

88.1 ns ± 0.101 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa394b1ca60>
├── 'a' --> tensor([[-1.3987,  0.2590, -2.8321],
│                   [ 0.8653,  1.9972, -0.2559]])
└── 'x' --> <FastTreeValue 0x7fa394b1cb20>
    └── 'c' --> tensor([[-2.3509,  0.3106, -0.5697, -0.8740],
                        [ 0.4058, -0.3023, -0.5851, -0.6074],
                        [-0.6653, -0.5996, -1.4165,  1.1686]])

In [7]:
%timeit t.a = new_value

89.7 ns ± 0.051 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2846,  0.5964, -1.3886],
               [-1.2387, -0.3852,  0.3116]]),
    x: Batch(
           c: tensor([[-2.3509,  0.3106, -0.5697, -0.8740],
                      [ 0.4058, -0.3023, -0.5851, -0.6074],
                      [-0.6653, -0.5996, -1.4165,  1.1686]]),
       ),
)

In [10]:
b.a

tensor([[-0.2846,  0.5964, -1.3886],
        [-1.2387, -0.3852,  0.3116]])

In [11]:
%timeit b.a

83.2 ns ± 0.122 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.3130,  3.2383, -0.6149],
               [ 0.2540, -1.8843, -0.0151]]),
    x: Batch(
           c: tensor([[-2.3509,  0.3106, -0.5697, -0.8740],
                      [ 0.4058, -0.3023, -0.5851, -0.6074],
                      [-0.6653, -0.5996, -1.4165,  1.1686]]),
       ),
)

In [13]:
%timeit b.a = new_value

662 ns ± 0.208 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 0.51 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

184 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

176 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa2bf1a7910>
├── 'a' --> tensor([[[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]],
│           
│                   [[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]],
│           
│                   [[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]],
│           
│                   [[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]],
│           
│                   [[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]],
│           
│                   [[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]],
│           
│                   [[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]],
│           
│                   [[-0.2846,  0.5964, -1.3886],
│                    [-1.2387, -0.3852,  0.3116]]])
└── 'x' --> <FastTreeValue 0x7fa395b87dc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.7 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa2bf1842b0>
├── 'a' --> tensor([[-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116],
│                   [-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116],
│                   [-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116],
│                   [-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116],
│                   [-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116],
│                   [-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116],
│                   [-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116],
│                   [-0.2846,  0.5964, -1.3886],
│                   [-1.2387, -0.3852,  0.3116]])
└── 'x' --> <FastTreeValue 0x7fa394b17df0>
    └── 'c' --> tensor([[-2.3509,  0.3106, -0.5697, -0.8740],
                        [ 0.4058, -0.3023, -0.5851, -0.6074],
                 

In [23]:
%timeit t_cat(trees)

38.3 µs ± 3.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.1 µs ± 92.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.3509,  0.3106, -0.5697, -0.8740],
                       [ 0.4058, -0.3023, -0.5851, -0.6074],
                       [-0.6653, -0.5996, -1.4165,  1.1686]],
              
                      [[-2.3509,  0.3106, -0.5697, -0.8740],
                       [ 0.4058, -0.3023, -0.5851, -0.6074],
                       [-0.6653, -0.5996, -1.4165,  1.1686]],
              
                      [[-2.3509,  0.3106, -0.5697, -0.8740],
                       [ 0.4058, -0.3023, -0.5851, -0.6074],
                       [-0.6653, -0.5996, -1.4165,  1.1686]],
              
                      [[-2.3509,  0.3106, -0.5697, -0.8740],
                       [ 0.4058, -0.3023, -0.5851, -0.6074],
                       [-0.6653, -0.5996, -1.4165,  1.1686]],
              
                      [[-2.3509,  0.3106, -0.5697, -0.8740],
                       [ 0.4058, -0.3023, -0.5851, -0.6074],
                       [-0.6653, -0.5996, -1.4165,  1.1686]],

In [26]:
%timeit Batch.stack(batches)

98.5 µs ± 60.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.3509,  0.3106, -0.5697, -0.8740],
                      [ 0.4058, -0.3023, -0.5851, -0.6074],
                      [-0.6653, -0.5996, -1.4165,  1.1686],
                      [-2.3509,  0.3106, -0.5697, -0.8740],
                      [ 0.4058, -0.3023, -0.5851, -0.6074],
                      [-0.6653, -0.5996, -1.4165,  1.1686],
                      [-2.3509,  0.3106, -0.5697, -0.8740],
                      [ 0.4058, -0.3023, -0.5851, -0.6074],
                      [-0.6653, -0.5996, -1.4165,  1.1686],
                      [-2.3509,  0.3106, -0.5697, -0.8740],
                      [ 0.4058, -0.3023, -0.5851, -0.6074],
                      [-0.6653, -0.5996, -1.4165,  1.1686],
                      [-2.3509,  0.3106, -0.5697, -0.8740],
                      [ 0.4058, -0.3023, -0.5851, -0.6074],
                      [-0.6653, -0.5996, -1.4165,  1.1686],
                      [-2.3509,  0.3106, -0.5697, -0.8740],
                   

In [28]:
%timeit Batch.cat(batches)

180 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

469 µs ± 29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
